In [149]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import codecs, json
import unicodedata
# pip install Unidecode  <OR> conda install Unidecode
import unidecode
import collections
import datetime

In [156]:
## Variables and what they mean
#   duplicate_dict = all duplicates
#   dict_removed_single_entries = all duplicates but removed single entries
#   duplicate_ids_kept = array of all ids from dict_removed_single_entries
#   dict_duplicate_compare_team_members = map values from teams to einstaklingsid
#   dict_name_entries = map values from member down one step (name->birthday->values) now (name+birthday->values)

# Import CSV

In [2]:
#importing all csv files
domarar = pd.read_csv('csv/blak-domarar.csv', sep=';', header=0)
einstaklingar = pd.read_csv('csv/blak-einstaklingar.csv', sep=';', header=0)
forsvarsmenn = pd.read_csv('csv/blak-forsvarsmenn.csv', sep=';', header=0)
lid = pd.read_csv('csv/blak-lid.csv', sep=';', header=0)
lidimoti = pd.read_csv('csv/blak-lidimoti.csv', sep=';', header=0)
lidsmenn = pd.read_csv('csv/blak-lidsmenn.csv', sep=';', header=0)
lidsstjorar = pd.read_csv('csv/blak-lidsstjorar.csv', sep=';', header=0)
thjalfarar = pd.read_csv('csv/blak-thjalfarar.csv', sep=';', header=0)
mot = pd.read_csv('csv/blak-mot.csv', sep=';', header=0)

# drop all SyndarLids with an ID (SyndarlidID)
# (the reason for not dropping using SyndarLid is because I don't trust that column to be inserted correctly with [0,1])
lid = lid[lid['SyndarlidID'].isna()]
# then dropping those two columns because we don't want virtual teams
lid = lid.drop(columns=['SyndarLid', 'SyndarlidID'])

# All duplicated birthdays
duplicated_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Nafn', 'Fdagur', 'Kyn'], keep=False)]
duplicated_fdagur_kyn_einstaklingar = einstaklingar[einstaklingar.duplicated(subset=['Fdagur', 'Kyn'], keep=False)]


# Filter duplicates by name and birthday

In [3]:
# Add all entries that have duplicated birthdays, then filter that to first_name->birthday-><people entries>
duplicate_dict = defaultdict(dict)
for index, row in duplicated_fdagur_kyn_einstaklingar.iterrows():
    full_name = row['Nafn']
    #only get the first part of full name 
    first_name = full_name.split()[0]
    # make first name lowercase
    first_name_lowercase = first_name.lower()
    # encode icelandic letters to english
    first_name_to_english = unidecode.unidecode(first_name_lowercase)
    # split birthday into year month and day and ignore second part (sec, min, hour)
    Fdagur_date = row['Fdagur'].split()[0]
    
    if first_name_to_english in duplicate_dict.keys():
        if Fdagur_date in duplicate_dict[first_name_to_english].keys():
            #if first name and Fdagur (birthday) exist in dict then append to that key (birthday)
            duplicate_dict[first_name_to_english][Fdagur_date].append(row.values)
        else:
            #if first name exists but Fdagur (birthday) does not exist in dict
            duplicate_dict[first_name_to_english][Fdagur_date] = [row.values]
    else:
        #if Fdagur (birthday) does not exist in dict
        duplicate_dict[first_name_to_english][Fdagur_date] = [row.values]
        

# Remove all single birthday entries (since that is not a duplicate)

In [4]:
# Remove all single birthday entries that are not duplicates
dict_removed_single_entries = defaultdict(dict)

for key, values in duplicate_dict.items():
    # key = nafn ('ludvik')
    for birthday, arrays in dict(values).items():
        # only get duplicates that there exists 2 or more entries for a birthday
        if(len(arrays) > 1):
            # used for when joining teams table
            if key in dict_removed_single_entries.keys():
                if birthday in dict_removed_single_entries[key].keys():
                    dict_removed_single_entries[key][birthday].append(arrays)
                else:
                    #if first name exists but Fdagur (birthday) does not exist in dict
                    dict_removed_single_entries[key][birthday] = arrays
            else:
                dict_removed_single_entries[key][birthday] = arrays

#dict_removed_single_entries

# Get all ids that exists in duplication

In [5]:
# get all ids in dict_removed_single_entries
duplicate_ids_kept = []
for key, values in dict_removed_single_entries.items():
    # key = nafn ('ludvik')
    for birthday, arrays in dict(values).items():
        for item in arrays:
            duplicate_ids_kept.append(item[0])

# Map teams table values to einstaklingsID

In [8]:
# Checking if two names are the same person
dict_duplicate_compare_team_members = defaultdict(dict)
for index, row in lidsmenn.iterrows():
    ids = row["EinstID"]
    if ids in duplicate_ids_kept:
        # now we only view ids that exist for duplicated people
        #print(ids)
        if ids in dict_duplicate_compare_team_members.keys():
            dict_duplicate_compare_team_members[ids].append(row.values)
        else:
            dict_duplicate_compare_team_members[ids] = [row.values]


## Map one step down (name->birthday -> value) now (name+birthday -> value)

In [10]:
dict_name_entries = {}
for key, value in dict_removed_single_entries.items():
    #get key and arrays for each person
    for birthday, arrays in dict(value).items():
        #get each array for person
        #print("KEY: " + key + " BIRTHDAY: " + birthday)
        new_key = key +"-"+ birthday
        for item in arrays:
            if new_key in dict_name_entries.keys():
                dict_name_entries[new_key].append(item[0])
            else:
                dict_name_entries[new_key] = [item[0]]
#dict_name_entries

## EinstaklingsID+birthday connected to all his data from teams table 

In [159]:
dict_einstaklingar_teammember_info = {}
for key, value in dict_name_entries.items():
    #print("<key>" + str(key) + " <value> " + str(value))
    for item in value:
        #print(item)
        if item in dict_duplicate_compare_team_members.keys():
            #print("<key>" + str(key) + " <item> " + str(item))
            for compare_arrays in dict_duplicate_compare_team_members[item]:
                mot_id = compare_arrays[0]
                lid_id = compare_arrays[1]
                player_id = compare_arrays[2]
                date_played = compare_arrays[3].split()[0]
                
                temp = (str(compare_arrays[3]) + " " + str(player_id) + " " + str(lid_id))   
                if key in dict_einstaklingar_teammember_info.keys():
                    dict_einstaklingar_teammember_info[key].append(temp)
                else:
                    dict_einstaklingar_teammember_info[key] = [temp]
        #print("xxxxxxxxxxxx")
#dict_einstaklingar_teammember_info

## Find if a potential duplicated person played two games at the same time in different teams (then he is not a duplication)

In [154]:
not_the_same_person = {}

def getTimeDifferenceFromNow(TimeStart, TimeEnd):
    timeDiff = TimeEnd - TimeStart
    return timeDiff.total_seconds() / 60

def find_duplicates(key, nums):
    num_set = set()
    duplicates = set()
    no_duplicate = -1
    sorted_nums = sorted(nums)
    
    for i in range(len(sorted_nums)):
        for j in range(i+1, len(sorted_nums)):
            
            # team one split
            sort_1 = sorted_nums[i].split()
            date_1 = sort_1[0]
            einstaklings_id_1 = sort_1[2]
            team_id_1 = sort_1[3]
            
            #hour minutes seconds
            time_1 = sort_1[1]
            time_1_split = time_1.split(":")
            time_1_hour = time_1_split[0]
            time_1_min = time_1_split[1]
            time_1_sec_split = time_1_split[2].split(".")
            time_1_sec = time_1_sec_split[0]
            time_1_micro = time_1_sec_split[1]
            #print(time_1_micro)
            time_1_final = datetime.time(int(time_1_hour), int(time_1_min), int(time_1_sec))
            
            
            # team two split
            sort_2 = sorted_nums[j].split()
            date_2 = sort_2[0]
            einstaklings_id_2 = sort_2[2]
            team_id_2 = sort_2[3]
            
            #hour minutes seconds
            time_2 = sort_2[1]
            time_2_split = time_2.split(":")
            time_2_hour = time_2_split[0]
            time_2_min = time_2_split[1]
            time_2_sec_split = time_2_split[2].split(".")
            time_2_sec = time_2_sec_split[0]
            time_2_micro = time_2_sec_split[1]
            time_2_final = datetime.time(int(time_2_hour), int(time_2_min), int(time_2_sec))
            
            
            #time_diff = (time_2_final - time_1_final).time()
            
            if((date_1 == date_2) and (team_id_1 != team_id_2)):
                print("=======================")
                print("Time Diff: " + str(time_2_final))
                #print("Time diff: " + time_diff)
                print("Key: " + key)
                print("Row 1: ")
                print(sorted_nums[i])
                print("Row 2: ")
                print(sorted_nums[j]);
                print("END OF THESE ROWS")
                
    
for key, value in dict_einstaklingar_teammember_info.items():
    #print(value)
    find_duplicates(key, value)

Time Diff: 22:54:28
Key: ana-1972-01-29
Row 1: 
2012-04-11 13:50:36.313 2065 4155
Row 2: 
2012-04-11 22:54:28.157 2065 1840
END OF THESE ROWS
Time Diff: 14:03:03
Key: anna-1972-05-30
Row 1: 
2005-03-29 13:56:01.373 1766 2226
Row 2: 
2005-03-29 14:03:03.560 1768 2227
END OF THESE ROWS
Time Diff: 18:01:22
Key: berglind-1975-10-31
Row 1: 
2010-04-15 14:45:54.687 3104 4448
Row 2: 
2010-04-15 18:01:22.030 3119 4450
END OF THESE ROWS
Time Diff: 11:09:30
Key: bragi-1931-04-26
Row 1: 
2007-01-05 11:01:51.687 1852 132
Row 2: 
2007-01-05 11:09:30.640 541 2873
END OF THESE ROWS
Time Diff: 17:48:28
Key: bryndis-1976-08-13
Row 1: 
2010-04-15 14:53:52.967 3107 4448
Row 2: 
2010-04-15 17:48:28.827 3113 4449
END OF THESE ROWS
Time Diff: 17:49:36
Key: charlotta-1978-02-21
Row 1: 
2010-04-15 14:46:45.247 3105 4448
Row 2: 
2010-04-15 17:49:36.890 3114 4449
END OF THESE ROWS
Time Diff: 22:10:12
Key: erla-1976-02-10
Row 1: 
2014-04-01 22:03:38.327 3654 6286
Row 2: 
2014-04-01 22:10:12.233 3654 6285
END OF 

In [ ]:
#reverted_back_to_dict = dict(duplicate_dict)
#reverted_back_to_dict
#file_path = "json/einstaklingar_map.txt" ## your path variable
#duplicate_dict_json = json.dump(duplicate_dict, codecs.open(file_path, 'w', encoding='utf-8'), separators=(';', ':'), sort_keys=True, indent=4) ### this saves the array in .json format
#json_obj = json.dumps(duplicate_dict, indent = 4)
#dumped = json.dumps(duplicate_dict, cls=NumpyEncoder)
#dumped
#pd.DataFrame(reverted_back_to_dict).to_csv(file_path, encoding='utf-8-sig')
#duplicate_dict_json = json.dump(reverted_back_to_dict, codecs.open(file_path, 'w', encoding='utf-8-sig'))

#json = json.dumps(reverted_back_to_dict)
#f = open(file_path,"w")
#f.write(str(reverted_back_to_dict))
#f.close()

=====================================================================================
=

In [ ]:
#FINAL STEP (run after everything is done):

#duplicated people put into it's own csv to be browsed later
pd.DataFrame(duplicated_einstaklingar).to_csv("csv/new/duplicated-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(duplicate_dict).to_csv("json/duplicate-map.json", encoding='utf-8-sig')


#save as new csv inside csv/new
pd.DataFrame(domarar).to_csv("csv/new/blak-domarar.csv", encoding='utf-8-sig')
pd.DataFrame(einstaklingar).to_csv("csv/new/blak-einstaklingar.csv", encoding='utf-8-sig')
pd.DataFrame(forsvarsmenn).to_csv("csv/new/blak-forsvarsmenn.csv.csv", encoding='utf-8-sig')
pd.DataFrame(lid).to_csv("csv/new/blak-lid.csv", encoding='utf-8-sig')
pd.DataFrame(lidimoti).to_csv("csv/new/blak-lidimoti.csv", encoding='utf-8-sig')
pd.DataFrame(lidsmenn).to_csv("csv/new/blak-lidsmenn.csv", encoding='utf-8-sig')
pd.DataFrame(lidsstjorar).to_csv("csv/new/blak-lidsstjorar.csv", encoding='utf-8-sig')
pd.DataFrame(mot).to_csv("csv/new/blak-mot.csv", encoding='utf-8-sig')
pd.DataFrame(thjalfarar).to_csv("csv/new/blak-thjalfarar.csv", encoding='utf-8-sig')